In [4]:
# Dependencies and set up
import pandas as pd
import numpy as np
import biom
from biom import load_table
table = load_table('/Users/karlaflores/Desktop/Git/Project-03/ITS_otu_table_wTax_20000_wMeta_extended_barebones2.biom') 

In [5]:
table

51965 x 1808 <class 'biom.table.Table'> with 1555951 nonzero entries (1% dense)

In [6]:
df = table.to_dataframe()
df

,511.O,242.O,445.O,574.O,919.O,841.O,737.O,323.O,652.O,991.O,...,1035.I,925.I,277.I,192.I,65.I,1082.I,804.I,741.I,986.I,1202.I
OTU_5,4379.0,0.0,4.0,175.0,5066.0,4.0,3.0,76.0,2504.0,35.0,...,0.0,52.0,27.0,48.0,0.0,0.0,115.0,114.0,241.0,0.0
OTU_337,0.0,3956.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
OTU_39829,1.0,11.0,194.0,0.0,5.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_33643,403.0,0.0,0.0,4068.0,0.0,1.0,5.0,264.0,5084.0,74.0,...,0.0,16.0,0.0,0.0,0.0,0.0,0.0,37.0,2.0,0.0
OTU_17891,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OTU_50872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_55438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_54018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_61959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
